# Building Your First Container

You've learned the core concepts of Docker on the learning platform and successfully installed Docker Desktop. This notebook is where you'll put that theory into practice. We will be working with the **data formatter** project, which you first saw in the GitHub Actions lesson, and we will containerise it to make it a portable, reproducible application.

To get the most out of this tutorial, we encourage you to **type out each command yourself**, rather than copy-pasting. This helps to build muscle memory and makes the commands feel natural over time.

---
## 1.&nbsp; Preparing Your Project for Docker 📂

Our first step is to add the most important file to our project: the `Dockerfile`. This is the recipe that tells Docker exactly how to build our Image.

1.  Open your existing `python-data-formatter` project folder in VS Code.
2.  In the VS Code Explorer, create a new file in the root of your project. Name it `Dockerfile` (with a capital 'D' and no file extension).
3.  Copy the following instructions into your new `Dockerfile`. We will break down what each line means.

In [ ]:
# Layer 1: Use an official Python image as a base image
FROM python:3.11-slim

# Layer 2: Set the working directory inside the container
WORKDIR /app

# Layers 3&4: Copy the requirements file and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Layers 5&6: Copy local project files into the container
# The first path is the source on your local machine.
# The second path is the destination inside the container's /app directory.
COPY src/ src/
COPY data/ data/

# Layer 7: Specify the command to run on container start
CMD ["python", "src/formatter.py", "--input", "data/raw_sales_data.csv", "--output", "data/cleaned_sales_data.csv"]

`FROM python:3.11-slim`: Every `Dockerfile` must start with a `FROM` instruction. It specifies the base Image to build upon. Here, we're using an official, lightweight version of Python 3.11 from Docker Hub.

`WORKDIR /app`: This sets the working directory for any subsequent commands. It's like running `cd /app` inside the container.

`COPY requirements.txt .`: This copies our local `requirements.txt` file into the `/app` directory inside the container.

`RUN pip install ...`: The `RUN` command executes shell commands. Here, we're installing our Python dependencies. The `--no-cache-dir` flag is a good practice that keeps the final Image smaller.
> Docker builds your image step-by-step, creating a "layer" for each instruction. When you rebuild an image, Docker checks if a layer has changed. If it hasn't, it reuses the layer from a cache instead of running the step again.
>
> This is why we copy requirements.txt and install dependencies before copying our src code. Our dependencies change much less often than our code. As long as requirements.txt doesn't change, Docker will use the cached layer with all our libraries pre-installed, making our builds much faster!

`COPY src/ src/`: This copies our entire local `src` directory into a `src` directory inside the container (i.e., `/app/src`).
> Here we've purposefully copied two folders, not the whole project. You can also use a `.dockerignore` file to exclude folders and files.


`CMD [...]`: This specifies the default command to execute when a container is run from this Image. Each item in the list is treated as a separate argument, so this is equivalent to running `python src/formatter.py --input data/raw_sales_data.csv --output data/cleaned_sales_data.csv` in the terminal.

---
## 2.&nbsp; The Docker Workflow: Build, Run, Repeat! 🔄

Now that we have our recipe, we can follow the core Docker workflow. This section will guide you through building your Image and running it as a Container.

### 2.1. Start Docker Desktop

Before we can issue any commands, we need to make sure Docker's core engine is running. On macOS and Windows, the Docker Desktop application is responsible for this.

1.  Find and open the Docker Desktop application on your computer.
2.  Wait for it to fully load. You'll know it's ready when the Docker icon in your system tray (or menu bar) is stable and the application window shows a green status indicator, often in the bottom-left corner.

> #### Why is this step necessary?
>
> Docker has two main parts: the **command-line client** (what you type in the terminal) and the **Docker Daemon** (the background engine that does the actual work). Docker Desktop starts and manages this daemon for you. If it isn't running, your terminal commands have no engine to connect to and will fail. Think of it like trying to drive a car without starting the engine first.

### 2.2. Build the Image

1.  Open the integrated terminal in VS Code (if it's not already open).
2.  Run the `docker build` command. This command looks for a `Dockerfile` in the current directory (`.`) and builds an Image from it. The `-t` flag allows us to 'tag', or name, our image.

In [ ]:
docker build -t data-formatter-app .

You will see Docker execute each step from your `Dockerfile` one by one. This might take a moment the first time as it downloads the base Python image.

### 2.3. Check Your Work: View Local Images

Now that the build is complete, let's confirm that our Image was created successfully. We can do this in two ways.

**1. Using the Command Line**

In your terminal, run the following command:

In [ ]:
docker images

You should see your new `data-formatter-app` image listed at the top of the output.

**2. Using Docker Desktop (The Visual Check)**

The command line is essential for automation, but the Docker Desktop app is perfect for visualising what you've just done.

* Open the Docker Desktop application.
* In the sidebar on the left, click on the **Images** tab.
* You will see your `data-formatter-app` image listed there, confirming it's ready to be used. This visual check is a great way to build confidence that your commands are working as expected.

### 2.3. Run the Container

Now that we have our Image, we can run it as a Container.

In [ ]:
docker run data-formatter-app

Success! Docker started a new container from your image, which executed the `CMD` from your `Dockerfile`. You should see the output from your `formatter.py` script:

```
Loading data from data/raw_sales_data.csv...
Cleaning data...
Saving cleaned data to data/cleaned_sales_data.csv...
Done.
```

The container completed its task and then stopped. But where is the cleaned_sales_data.csv file? You won't find it in your project's data folder. Let's find out why.

---
## 3.&nbsp; Getting Data In and Out of Containers ↔️

The script ran perfectly, but our output file seems to have vanished. This isn't a bug; it's a core feature of Docker called container isolation. By default, a container's filesystem is a temporary, sealed-off workspace. When the container stops, its entire filesystem is deleted, taking our output file with it.

To make our formatter useful, we need a way to connect a folder on our computer (the host) to a folder inside the container. This is done with a volume.

### 3.1. Retrieving Output with a Volume

A Docker **volume** (specifically, a "bind mount") creates a live, two-way portal between a folder on your host machine and a folder inside the container. Let's use one to retrieve our output file.

1.  In your terminal, run the `docker run` command again, but this time add the `-v` (for volume) flag to link your local `data` folder to the container's `/app/data` folder.

    ```bash
    # For macOS/Linux:
    docker run -v "$(pwd)/data:/app/data" data-formatter-app

    # For Windows Command Prompt:
    # docker run -v "%cd%/data:/app/data" data-formatter-app
    ```

    > `$(pwd)` on macOS/Linux and `%cd%` on Windows are terminal shortcuts for your current working directory. This is a robust way to make sure the path is always correct.

2.  The script will run just as before. But now, check your project's `data` folder on your computer. The `cleaned_sales_data.csv` file is there\!

    By creating a volume, you tell Docker to use your local data folder as the container's /app/data folder. They become the same place. The container's script has no idea it's writing to the outside world; it just saves the file to /app/data, and because of the live link you created, the file instantly appears on your computer.

### 3.2. Processing New Files on the Fly

We've solved the output problem, but our container is still inflexible. The `CMD` in our `Dockerfile` is hard-coded to always process `raw_sales_data.csv`. What if we have a new file?

**The `CMD` is just a default**. We can override it by adding a new command to the end of our `docker run` statement. This transforms our image from a single-use script into a flexible tool.

1.  First, let's create a new data file to process. Create a new file inside your local `data` folder named `new_batch.csv` and add the following content:

    ```csv
    order_id,product_name,quantity,unit_price,order_date
    2001,SuperTool,5,99.99,2025-02-01
    2002,MegaFrame,2,450.00,2025-02-01
    ```

2.  Now, run the container again. We will use the same `-v` flag, but this time we will add a new command at the end to tell our script which input and output files to use.

    ```bash
    # For macOS/Linux:
    docker run -v "$(pwd)/data:/app/data" data-formatter-app python src/formatter.py --input data/new_batch.csv --output data/cleaned_new_batch.csv

    # For Windows Command Prompt:
    # docker run -v "%cd%/data:/app/data" data-formatter-app python src/formatter.py --input data/new_batch.csv --output data/cleaned_new_batch.csv
    ```

This time, Docker ignores the default `CMD` and runs your new command instead. Check your `data` folder one last time. You will now see `cleaned_new_batch.csv` sitting right next to your other files.

This is the core power of Docker: build the image once, run it many times with different data and commands. You've successfully created a portable, reusable data-processing tool.

---
## 4.&nbsp; Managing Containers and Cleaning Up 🧹

Running containers is just the beginning. You also need to know how to see what's running and how to clean up afterwards to keep your system tidy.

### 4.1. Listing Containers

The `docker ps` command is your main tool for viewing containers. By default, it only shows currently running containers.

In [ ]:
docker ps

This command will likely show an empty list because our container stopped as soon as the script finished. To see all containers, including stopped ones, use the `-a` flag.

In [ ]:
docker ps -a

Now you will see the container you just ran listed, along with its unique ID and a randomly assigned name (like `pensive_mclean`).

> **Visual Check:** In Docker Desktop, click the **Containers** tab in the sidebar. You will see the same stopped container listed there. You can click on it to view its logs or other details.

### 4.2. Tidying Up: Removing Containers and Images

Stopped containers and old images can quickly use up disk space. It's a good habit to clean them up regularly.

1.  **Remove the stopped container:** Use the `docker rm` command followed by the Container ID or name from the `docker ps -a` output.

In [ ]:
docker rm <CONTAINER_ID_OR_NAME>

> Tip: You only need to type the first few unique characters of the ID; you don't need the full string.

2.  **Remove the image:** Once all containers using an image are removed, you can delete the image itself with `docker rmi`.

In [ ]:
docker rmi data-formatter-app

### 4.3. The Professional's Cleanup Command

Removing items one by one is fine, but there's a much faster way to tidy up everything you aren't currently using. The `docker system prune` command is a powerful tool for this.

> **Warning:** This command is very effective and will remove all stopped containers, all networks not used by at least one container, and all dangling images. Use it when you're ready for a full clean-up.

In [ ]:
docker system prune

Docker will ask you to confirm. Type `y` and press Enter. It will then report everything it has reclaimed.

This command is the standard way to keep your local Docker environment clean and efficient.

---
## 5.&nbsp; Advanced `docker run` Options 🚀

You've now mastered the fundamental Docker workflow. Let's look at two powerful flags for the `docker run` command that will make your development and debugging process more efficient.

First, let's build our image again:

In [ ]:
docker build -t data-formatter-app .

### 5.1. Auto-Cleanup with `--rm`

In the last section, you learned how to list stopped containers and then remove them with `docker rm`. For one-off tasks like our formatter script, having to clean up the container manually every time can become tedious.

We can instruct Docker to remove the container automatically as soon as it finishes its task by adding the `--rm` flag.

1.  Run the command again, this time including `--rm`.

    ```bash
    # For macOS/Linux:
    docker run --rm -v "$(pwd)/data:/app/data" data-formatter-app

    # For Windows Command Prompt:
    # docker run --rm -v "%cd%/data:/app/data" data-formatter-app
    ```

2.  After the script's output appears, check your list of containers.

In [ ]:
docker ps -a

You'll see that the container you just ran is not in the list. It was automatically removed upon completion. This is the standard way to run "fire-and-forget" tasks and keep your system tidy.

### 5.2. Debugging with an Interactive Session (`-it`)

Sometimes your script will fail, and you will need to understand why. Perhaps a file path is wrong, and you want to explore the container's filesystem to debug the issue.

You can start a container with an interactive shell using the `-it` flags. This gives you a command prompt inside the container itself.

1.  Run the following command. Instead of letting the container run the default `CMD`, we will override it and tell it to run `/bin/sh`, which is a basic shell program.

In [ ]:
docker run --rm -it data-formatter-app /bin/sh

`-i` (interactive) keeps the input channel open, and `-t` allocates a terminal. You will almost always use them together as `-it`.

2.  Your terminal prompt will change to something like `/app #`. You are now executing commands inside the running container. Let's look around:

      * Run `ls -la` to see the files in the `/app` directory. You will see `src/`, `data/`, and `requirements.txt`, just as our `Dockerfile` specified.
      * Run `pwd` to confirm the working directory is `/app`.

3.  When you are finished exploring, simply type `exit` and press Enter.

    Because we included the `--rm` flag, the container is removed the moment you exit the interactive session.

Let's clean up again by deleting all containers (there should be none though) as well as by deleting the unused image.

In [ ]:
docker system prune

In [ ]:
docker rmi data-formatter-app

> Note: `docker system prune` allows for deleting all unused images automatically by adding the `-a` flag.
>
> However, since this removes **all** images, it is usually not recommended.

---
## 6.&nbsp; Challenges 😀

Congratulations! You've successfully built a `Dockerfile`, used it to create a custom Image, and run your application in an isolated, reproducible Container. This is the fundamental workflow that underpins modern MLOps and software deployment.

Your final task is to apply these skills to the `sentiment-analysis-project` you have been working on throughout this project.

Your challenge is to create a new `Dockerfile` inside your `sentiment-analysis-project` folder. This workflow should containerise the `predict.py` script, allowing you to get a sentiment prediction from a container.

Think about which instruction in the `Dockerfile` you need to change. How do you pass command-line arguments?

The workflow is the same: **Edit** your `Dockerfile`, **Build** the new image, and **Run** the container to see the prediction output. Good luck!